In [17]:
import pandas as pd
import json
from datetime import datetime
import os

In [18]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 

# cleaning data

In [20]:
import re
df = pd.read_csv('goodreads_output3.csv')
dfwin = pd.read_csv('AwardDates.csv')
data = dict()
with open('dates.json') as json_file:
    data = json.load(json_file)
for d in data: 
    date = d['date']
    result = re.search('Published(.*)', date)
    partial = result.group(1)
    if('by' in partial) :
        partial = re.search('(.*)by', partial)
        d['date'] = partial.group(1).lstrip().rstrip()
    else:
        d['date'] = partial.lstrip().rstrip()

with open('new.json', 'w') as new:
    json.dump(data, new)

In [24]:
# calculating dates and matching isbn
data = dict()
with open('new.json') as json_file:
    data = json.load(json_file)
hi = my_dictionary()
h2 = my_dictionary()
h3 = list()
for d in data:

    ds = str(d['date']).split()
    if len(ds)>1:
        ds[1]=ds[1].replace('th','').replace('st','').replace('rd','').replace('nd','')
        d['date'] = ''.join(str(e)+' ' for e in ds)
    if len(ds)>3:
         d['date'] = ''.join([ds[0],' ',ds[1],' ',ds[2],' '])
    try:
        d['date']=datetime.strptime(d['date'], '%B %d %Y ')
    except ValueError :
        try:
           d['date']= datetime.strptime(d['date'], '%B %Y ')
        except ValueError :
           d['date']= datetime.strptime(d['date'], '%Y')
    if(df[df['url']==d['url']].empty):
        continue
    year=df[df['url']==d['url']]['year'].iloc[0]
    win=datetime.strptime(dfwin[dfwin['year'] ==year]['Winners Announced'].iloc[0],'%m/%d/%Y')
    if((win -  d['date']).days<1):
        d['date']=(win -  d['date']).days + 365
    else:
        d['date']=(win -  d['date']).days 
#     if (d['name'] == 'Calling Dr. Laura'):
#         print(d['date'])
#         print(win)
    hi.add(d['url'], d['date'])
    h2.add(d['url'],d['isbn'])
    h3.append(d['isbn'])
    


# merging data and adding 'times_appred_NYTime','avgrank_NYTime','bestRank','listnames' to are coulems

In [25]:
dfnew=pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
df.merge(dfnew,on='url', how='left').merge(dfnew1,on='url', how='left').to_csv("withdays.csv")
df = pd.read_csv('withdays.csv')
df = df.reindex(df.columns.tolist() + ['times_appred_NYTime','avgrank_NYTime','bestRank','listnames'], axis=1)


In [27]:
df['year']

0       2013
1       2013
2       2013
3       2013
4       2013
        ... 
3262    2016
3263    2016
3264    2016
3265    2016
3266    2016
Name: year, Length: 3267, dtype: int64

reading newYork Data and adding to feilds

In [30]:
newYork_dir = "newYork Data/"
data = dict()
data1 = dict()
for filename in os.listdir(newYork_dir):
    if filename.endswith(".json") and filename.startswith("res") and not(filename.endswith("10.json")): 
        cmplt_path = os.path.join(newYork_dir, filename)
        print(cmplt_path)
        with open(cmplt_path) as json_file :
            new_data = json.load(json_file)
            data.update(new_data)
    else:
        continue

print(len(data))
h4 = my_dictionary()
h5 = my_dictionary()
h6 = my_dictionary()
h7 = my_dictionary()
for i in h3:
    try:
        if d[i]['num_results']==0:
                df.loc[df['isbn']==i,['times_appred_NYTime']] = 0
                df.loc[df['isbn']==i,['avgrank_NYTime']] =float('inf')
                df.loc[df['isbn']==i,['bestRank']] = float('inf')
                df.loc[df['isbn']==i,['listnames']] = str(0)
                continue
        for j in data[i]['results']:
            bestrank=float('inf')
            num=0
            count = 0 
            listname = set()
            if len(j['ranks_history'])==0:
                df.loc[df['isbn']==i,['times_appred_NYTime']] =0
                df.loc[df['isbn']==i,['avgrank_NYTime']] =float('inf')
                df.loc[df['isbn']==i,['bestRank']] = float('inf')
                df.loc[df['isbn']==i,['listnames']] = str(0)
                continue
            year = df[df['isbn']==i]['year']
            win=datetime.strptime(dfwin[dfwin['year'] ==year]['Winners Announced'].iloc[0],'%m/%d/%Y')
            for z in j['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    num+= z['rank']
                    listname.add(z['list_name'])
                    if bestrank>z['rank']:
                        bestrank =z['rank']
                    count  = count + 1
            num=num/count
            df.loc[df['isbn']==i,['times_appred_NYTime']] = count
            df.loc[df['isbn']==i,['avgrank_NYTime']] =num
            df.loc[df['isbn']==i,['bestRank']] = bestrank
            df.loc[df['isbn']==i,['listnames']] = str(len(listname))
    except KeyError :
        df.loc[df['isbn']==i,['times_appred_NYTime']] = 0
        df.loc[df['isbn']==i,['avgrank_NYTime']] =float('inf')
        df.loc[df['isbn']==i,['bestRank']] = float('inf')
        df.loc[df['isbn']==i,['listnames']] = str(0)
        continue


newYork Data/res8.json
newYork Data/res4.json
newYork Data/res5.json
newYork Data/res9.json
newYork Data/res2.json
newYork Data/res3.json
newYork Data/res1.json
newYork Data/res6.json
newYork Data/res7.json
3350


KeyError: 'isbn'

saving data

In [22]:
df.to_csv("withdays.csv")

cheak nall values

In [23]:
df = pd.read_csv('withdays.csv')

def nans(df): return df[df.isnull().any(axis=1)]
nans(df)

,Unnamed: 0,Unnamed: 0.1,year,name,writer,category,winner,num 1 stars,num 2 stars,num 3 stars,...,average_rating_w,num_ratings,num_reviews,url,daystocom,isbn,times_appred_NYTime,avgrank_NYTime,bestRank,listnames
407,407,407,2014,Alpha & Omega,"['Joe Hill', 'Gabriel Rodríguez']",Graphic Novels & Comics,0,0.000000,0.003333,0.046667,...,4.03,658776.0,59771.0,https://www.goodreads.com/book/show/16164271-a...,NaN,NaN,NaN,NaN,NaN,NaN
666,666,666,2014,What If?: Serious Scientific Answers to Absurd...,['Randall Munroe'],Nonfiction,0,0.003356,0.000000,0.063758,...,4.16,132666.0,9829.0,https://www.goodreads.com/book/show/21413662-w...,NaN,NaN,NaN,NaN,NaN,NaN
778,778,778,2014,Make It Ahead: A Barefoot Contessa Cookbook,['Ina Garten'],Food & Cookbooks,1,0.016722,0.060201,0.200669,...,4.16,166255.0,1704.0,https://www.goodreads.com/book/show/20697389-m...,NaN,NaN,NaN,NaN,NaN,NaN
808,808,808,2015,"The Wicked + The Divine, Vol. 2: Fandemonium","['Kieron Gillen', 'Jamie McKelvie', 'Matt Wils...",Graphic Novels & Comics,0,0.000000,0.023490,0.124161,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/24666002-t...,NaN,NaN,NaN,NaN,NaN,NaN
979,979,979,2015,Angles of Attack,['Marko Kloos'],Science Fiction,0,0.000000,0.030000,0.230000,...,4.11,77265.0,4197.0,https://www.goodreads.com/book/show/23470865-a...,224.0,NaN,NaN,NaN,NaN,NaN
1206,1206,1206,2012,Clockworks,"['Joe Hill', 'Gabriel Rodríguez']",Graphic Novels & Comics,0,0.000000,0.012158,0.066869,...,4.03,659090.0,59783.0,https://www.goodreads.com/book/show/13490570-c...,NaN,NaN,NaN,NaN,NaN,NaN
1609,1609,1609,2017,"The Wicked + The Divine, Vol. 5: Imperial Phas...","['Kieron Gillen', 'Jamie McKelvie', 'Matt Wils...",Graphic Novels & Comics,0,0.000000,0.020067,0.103679,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/33585540-t...,NaN,NaN,NaN,NaN,NaN,NaN
1996,1996,1996,2018,"The Wicked + The Divine, Vol. 6: Imperial Phas...","['Kieron Gillen', 'Jamie McKelvie', 'Matt Wils...",Graphic Novels & Comics,0,0.013423,0.050336,0.174497,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/35651693-t...,NaN,NaN,NaN,NaN,NaN,NaN
2362,2362,2362,2018,A Court of Wings and Ruin,['Sarah J. Maas'],Best of the Best,0,0.024823,0.070922,0.049645,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/23766634-a...,217.0,1408857901,0.0,inf,inf,0.0
2429,2429,2429,2011,Keys to the Kingdom,"['Joe Hill', 'Gabriel Rodríguez']",Graphic Novels & Comics,0,0.000000,0.003356,0.073826,...,4.03,658776.0,59771.0,https://www.goodreads.com/book/show/9674335-ke...,NaN,NaN,NaN,NaN,NaN,NaN


read data from json 11 with different struct and saving to csv

In [31]:
with open('newYork Data/res10.json') as json_file:
    data = json.load(json_file)
for i in data:
    if data[i]['num_results']==0:
        df.loc[df['url']==i,['times_appred_NYTime']] = float('inf')
        df.loc[df['url']==i,['avgrank_NYTime']] =float('inf')
        df.loc[df['url']==i,['bestRank']] = float('inf')
        df.loc[df['url']==i,['listnames']] = str(0)
        continue
    for j in data[i]['results']:
        bestrank=float('inf')
        num=0
        listname = set()
        if len(j['ranks_history'])==0:
                df.loc[df['url']==i,['times_appred_NYTime']] = float('inf')
                df.loc[df['url']==i,['avgrank_NYTime']] =float('inf')
                df.loc[df['url']==i,['bestRank']] = float('inf')
                df.loc[df['url']==i,['listnames']] = str(0)
                continue
        year = df[df['url']==i]['year']
        for k in year:
            win=datetime.strptime(dfwin[dfwin['year'] ==k]['Winners Announced'].iloc[0],'%m/%d/%Y')
            for z in j['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    num+= z['rank']
                    listname.add(z['list_name'])
                    if bestrank>z['rank']:
                        bestrank =z['rank']
                    count  = count + 1
            num=num/count
            df.loc[(df['url']==i)&(df['year']==k),['times_appred_NYTime']] = count
            df.loc[(df['url']==i)&(df['year']==k),['avgrank_NYTime']] =num
            df.loc[(df['url']==i)&(df['year']==k),['bestRank']] = bestrank
            df.loc[(df['url']==i)&(df['year']==k),['listnames']] = str(len(listname))
df.to_csv("withdays.csv")


NameError: name 'count' is not defined

saving nulls in file

In [560]:
def nans(df): return df[df.isnull().any(axis=1)]
nans(df).to_csv("nulls.csv")



In [ ]:
# dfnew=pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
# dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
# # dfnew2 = pd.DataFrame(h5.items(), columns=['isbn', 'times_appred_NYTime'])
# # dfnew3 = pd.DataFrame(h4.items(), columns=['isbn', 'avgrank_NYTime'])
# # dfnew4 = pd.DataFrame(h6.items(), columns=['isbn', 'bestRank'])
# # dfnew5 = pd.DataFrame(h7.items(), columns=['isbn', 'listnames'])
# df.merge(dfnew,on='url', how='left').merge(dfnew1,on='url', how='left').to_csv("withdays.csv")
# # .merge(dfnew3,on='isbn', how='left').merge(dfnew4,on='isbn', how='left').merge(dfnew5,on='isbn', how='left').to_csv("withdays.csv")